In [1]:
import glob
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import numpy as np
from regions import Regions
from astropy.nddata import Cutout2D
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
from astropy.table import Table, vstack


f140m_nrca_catalog = Table.read('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/f140m_nrca_indivexp_merged_dao_after_merger_combined_with_satstars.fits')
f140m_nrcb_catalog = Table.read('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/f140m_nrcb_indivexp_merged_dao_after_merger_combined_with_satstars.fits')
f480m_nrca_catalog = Table.read('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/f480m_nrca_indivexp_merged_dao_after_merger_combined_with_satstars.fits')
f480m_nrcb_catalog = Table.read('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/f480m_nrcb_indivexp_merged_dao_after_merger_combined_with_satstars.fits')
f560w_catalog = Table.read('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/f560w_mirimage_indivexp_merged_dao_after_merger_combined_with_satstars_fixed.fits')

base_filters = ['f140m', 'f480m', 'f560w']

def filter_further_catalog(catalog, nmatch_th=2):
    nmatch = catalog['nmatch']
    from_sat_catalog = catalog['from_sat_catalog']
    filtered_idx = np.where((nmatch >= nmatch_th) | (from_sat_catalog == True))[0]
    return catalog[filtered_idx]

def merge_nrca_nrcb(filt, nmatch_th=4):
    nrca_catalog = Table.read(f'/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/{filt}_nrca_indivexp_merged_dao_after_merger_combined_with_satstars.fits')
    nrcb_catalog = Table.read(f'/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/{filt}_nrcb_indivexp_merged_dao_after_merger_combined_with_satstars.fits')

    nrca_catalog_filtered = nrca_catalog[nrca_catalog['nmatch'] >= nmatch_th]
    nrcb_catalog_filtered = nrcb_catalog[nrcb_catalog['nmatch'] >= nmatch_th]
    idx, d2d, d3d = nrca_catalog_filtered['skycoord'].match_to_catalog_sky(nrcb_catalog_filtered['skycoord'])

    nrca_skycoord = nrca_catalog_filtered['skycoord']
    nrcb_skycoord = nrcb_catalog_filtered['skycoord']
    match_f140m = d2d < 0.1*u.arcsec

    new_nrcb = np.setdiff1d(np.arange(len(nrcb_skycoord)), idx[match_f140m])

    merged_catalog = vstack([nrca_catalog_filtered, nrcb_catalog_filtered[new_nrcb]])
    return merged_catalog

def make_base_table(catalogs, filters):
    base_table = Table()
    base_table['skycoord_ra'] = catalogs[0]['skycoord'].ra
    base_table['skycoord_dec'] = catalogs[0]['skycoord'].dec
    base_skycoord = catalogs[0]['skycoord']
    for col in ['flux_fit', 'flux_err', 'nmatch', 'nmatch_good', 'qfit', 'cfit', 'sharpness', 'roundness1', 'roundness2', 'from_sat_catalog']:
        base_table[col+'_'+filters[0]] = catalogs[0][col]

    for i, catalog in enumerate(catalogs[1:]):
        idx, d2d, d3d = base_skycoord.match_to_catalog_sky(catalog['skycoord'])
        match = d2d < 0.1*u.arcsec
        new_entries = np.setdiff1d(np.arange(len(catalog)), idx[match])
        new_catalog = catalog[new_entries].copy()
        new_tab = Table()
        new_tab['skycoord_ra'] = new_catalog['skycoord'].ra
        new_tab['skycoord_dec'] = new_catalog['skycoord'].dec
        for col in ['flux_fit', 'flux_err', 'nmatch', 'nmatch_good', 'qfit', 'cfit', 'sharpness', 'roundness1', 'roundness2', 'from_sat_catalog']:
            new_tab[col+'_'+filters[i+1]] = new_catalog[col]
        base_table = vstack([base_table, new_tab])

    return base_table

def add_to_base_table(base_tab, catalogs, filters):
    base_skycoord = SkyCoord(ra=base_tab['skycoord_ra'], dec=base_tab['skycoord_dec'])
    for i, catalog in enumerate(catalogs):
        idx, d2d, d3d = base_skycoord.match_to_catalog_sky(catalog['skycoord'])
        match = d2d < 0.1*u.arcsec
        for col in ['flux_fit', 'flux_err', 'nmatch', 'nmatch_good', 'qfit', 'cfit', 'sharpness', 'roundness1', 'roundness2', 'from_sat_catalog']:
            matched_values = np.full(len(base_tab), np.nan)
            matched_values[match] = catalog[col][idx[match]]
            base_tab[col+'_'+filters[i]] = matched_values
    return base_tab

f140m_catalog = merge_nrca_nrcb('f140m')
f162m_catalog = merge_nrca_nrcb('f162m')
f182m_catalog = merge_nrca_nrcb('f182m')
f187n_catalog = merge_nrca_nrcb('f187n')

f210m_catalog = merge_nrca_nrcb('f210m')
f335m_catalog = merge_nrca_nrcb('f335m')
f360m_catalog = merge_nrca_nrcb('f360m')
f405n_catalog = merge_nrca_nrcb('f405n')
f410m_catalog = merge_nrca_nrcb('f410m')

f480m_catalog = merge_nrca_nrcb('f480m')
f770w_catalog = Table.read('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/f770w_mirimage_indivexp_merged_dao_after_merger_combined_with_satstars_fixed.fits')
f1000w_catalog = Table.read('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/f1000w_mirimage_indivexp_merged_dao_after_merger_combined_with_satstars_fixed.fits')
f1280w_catalog = Table.read('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/f1280w_mirimage_indivexp_merged_dao_after_merger_combined_with_satstars_fixed.fits')
f2100w_catalog = Table.read('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/f2100w_mirimage_indivexp_merged_dao_after_merger_combined_with_satstars_fixed.fits')  

f560w_catalog_filtered = filter_further_catalog(f560w_catalog, nmatch_th=2)
f770w_catalog_filtered = filter_further_catalog(f770w_catalog, nmatch_th=2)
f1000w_catalog_filtered = filter_further_catalog(f1000w_catalog, nmatch_th=2)
f1280w_catalog_filtered = filter_further_catalog(f1280w_catalog, nmatch_th=2)
f2100w_catalog_filtered = filter_further_catalog(f2100w_catalog, nmatch_th=2)


base_tables = make_base_table([f140m_catalog, f480m_catalog, f560w_catalog_filtered], ['f140m', 'f480m', 'f560w'])
final_table = add_to_base_table(base_tables, [f162m_catalog, f182m_catalog, f187n_catalog, f210m_catalog, f335m_catalog, f360m_catalog, f405n_catalog, f410m_catalog,
f770w_catalog_filtered, f1000w_catalog_filtered, f1280w_catalog_filtered, f2100w_catalog_filtered], 
['f162m', 'f182m', 'f187n', 'f210m', 'f335m', 'f360m', 'f405n', 'f410m', 'f770w', 'f1000w', 'f1280w', 'f2100w'])

print(len(final_table))
final_table.write('/orange/adamginsburg/w51/TaehwaYoo/jwst_w51/catalogs/final_catalog.fits', overwrite=True)

25940
